In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import brian2 as bb
from brian2 import ms, second, Hz, mV, pA, nS, pF
from matplotlib import pyplot as plt
from tqdm import tqdm
import assemblyseq.assemblyseq as asq

faster_run = True

if faster_run:
    bb.set_device('cpp_standalone', directory='PETH_standalone', build_on_run=False)

In [2]:
bb.start_scope()
if faster_run:
    bb.get_device().reinit()
    bb.get_device().activate(build_on_run=False, directory='PETH_standalone')

nn = asq.Nets(Ne=20000, Ni=5000, cp_ee=.01, cp_ie=.01, cp_ei=0.01, cp_ii=.01,
              n_ass=4, s_ass=500, pr=.15, pf=.03, symmetric_sequence=True, p_rev=.03,
              g_ee=0.1 * nS, g_ie=0.1 * nS, g_ei=0.4 * nS, g_ii=0.4 * nS)

initiated  Thu May  7 14:44:33 2020


In [3]:
nn.generate_ps_assemblies('gen_no_overlap')
nn.set_net_connectivity()

nn.set_spike_monitor()
nn.set_rate_monitor()

for gr_num in tqdm(range(nn.n_ass)):
    gr = nn.p_ass_index[0][gr_num]
    t_inp = (20.55 + gr_num * .1) * second
    nn.set_noisy_input(gr, t_inp, sigma=0 * ms)

nn.balance(5 * second, 5.)
nn.balance(5 * second, 1.)
nn.balance(5 * second, .1)
nn.balance(5 * second, .01)
nn.run_sim(22*second)
# nn.Pe.I -= .0 * pA
if faster_run:
    bb.device.build(directory='STDP_standalone', compile=True, run=True, debug=False)

A random sequence without overlaps is created


100%|██████████| 4/4 [00:00<00:00, 75.80it/s]

connections imprinted!  Thu May  7 14:44:40 2020


balanced:  0.4028899669647217
balanced:  0.16179513931274414
balanced:  0.20635628700256348
balanced:  0.1691117286682129
run:  0.17310690879821777


In [8]:
nn.Pe[2]

<Subgroup 'neurongroup_subgroup_2' of 'neurongroup' from 2 to 3>

In [5]:
gr_num

4